In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv("dataset/train.csv")

In [51]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
final_pieline=ColumnTransformer([
    ("Age",SimpleImputer(strategy='median'),["Age","Cabin","Ticket"]),
    ("Num",StandardScaler(),["Fare"]),
],
    remainder='passthrough')

In [52]:
def split_fnc(x):
    listt=x.split(" ")[:-1]
    string="".join(listt)
    for char in "/",".",",":
        string=string.replace(char,"")
    return string if string!="" else np.nan

In [53]:
def split_cabin(x):
    if x is np.nan:
        return
    listx=x.split(" ")
    for el in listx:
        if len(listx)==1:
            break
        if len(el)==1:
            listx.pop(listx.index(el))
    return listx[0][0]

In [54]:
def count_cabin(x):
    if x is np.nan:
        return
    listx=x.split(" ")
    for el in listx:
        if len(listx)==1:
            break
        if len(el)==1:
            listx.pop(listx.index(el))
    return len(listx)

In [55]:
def transform(dataframe,labled=True):
    copyd=dataframe.copy()
    indexes=copyd.pop("PassengerId")
    if labled:
        y=copyd.pop("Survived").astype("bool")
    copyd.Name=copyd[["Name"]].apply(lambda element: element.Name.split(",")[0],axis=1)
    copyd["sub-Ticket"] = copyd["Ticket"].apply(split_fnc)
    copyd["Ticket"]=copyd["Ticket"].apply(lambda x:x.split(" ")[-1] if x.split(" ")[-1].isnumeric() else np.nan)
    copyd["sub-Cabin"] = copyd["Cabin"].apply(split_cabin)
    copyd["Cabin"] = copyd["Cabin"].apply(count_cabin)
    copyd["Ticket"] = copyd["Ticket"].astype('float')
    copyd["Cabin"] = copyd["Cabin"].astype('float')
    for col in "Name","Embarked","Sex","sub-Cabin","sub-Ticket":
        copyd[col] = copyd[col].astype('category')
        copyd[col] = copyd[col].cat.codes.astype('category')
    X=final_pieline.fit_transform(copyd)
    return X,y if labled else indexes

In [56]:
X,y=transform(df)

In [57]:
from sklearn.ensemble import RandomForestClassifier
rf_class=RandomForestClassifier(bootstrap=True,max_features="auto",n_estimators=400)

In [58]:
from sklearn.model_selection import cross_val_predict
y_pred_amel=cross_val_predict(rf_class,X,y,cv=5)

In [59]:
from sklearn.metrics import precision_score, recall_score
precision_score(y, y_pred_amel),recall_score(y, y_pred_amel)

(0.8245033112582781, 0.7280701754385965)

In [88]:
rf_class=RandomForestClassifier(bootstrap=True,max_features="auto",n_estimators=400)
rf_class.fit(X,y)

RandomForestClassifier(n_estimators=400)

In [89]:
test=pd.read_csv("dataset/test.csv")

In [90]:
copy=test.copy()

In [91]:
imputer=SimpleImputer(strategy="median")

In [92]:
copy[['Fare']]=imputer.fit_transform(X=copy[['Fare']])

In [93]:
X_test,indexes=transform(copy,labled=False)

In [94]:
y_test_pred=rf_class.predict(X_test)

In [109]:
y_test_pred_serie=pd.Series(y_test_pred.astype("int"))

In [110]:
y_test_pred_serie

0      0
1      0
2      0
3      0
4      1
      ..
413    0
414    1
415    0
416    0
417    0
Length: 418, dtype: int32

In [111]:
output=pd.DataFrame.merge(indexes,y_test_pred_serie.rename("Survived"),right_index=True,left_index=True)

In [116]:
output.to_csv("submission/titanic.csv",index=False)